In [1]:
%reload_kedro
from pyspark.sql import SparkSession

2019-06-29 20:23:09,393 - root - INFO - ** Kedro project prospa
2019-06-29 20:23:09,396 - anyconfig - INFO - Loading: /Users/Emil_Pastor/Documents/Projects/Startup/myanswer/interview-test-data-engineer/prospa/conf/base/logging.yml
2019-06-29 20:23:09,401 - anyconfig - INFO - Loading: /Users/Emil_Pastor/Documents/Projects/Startup/myanswer/interview-test-data-engineer/prospa/conf/base/catalog.yml
2019-06-29 20:23:09,404 - anyconfig - INFO - Loading: /Users/Emil_Pastor/Documents/Projects/Startup/myanswer/interview-test-data-engineer/prospa/conf/base/credentials.yml
2019-06-29 20:23:09,406 - anyconfig - INFO - Loading: /Users/Emil_Pastor/Documents/Projects/Startup/myanswer/interview-test-data-engineer/prospa/conf/base/parameters.yml
2019-06-29 20:23:09,410 - root - INFO - Defined global variables proj_dir, proj_name, conf and io


# Load data sets

In [2]:
customer_dimension = io.load("customer_dimension")
part_dimension = io.load("part_dimension")
supplier_dimension = io.load("supplier_dimension")
order_fact = io.load("order_fact")

2019-06-29 20:23:09,426 - kedro.io.data_catalog - INFO - Loading data from `customer_dimension` (SparkDataSet)...
2019-06-29 20:23:14,613 - kedro.io.data_catalog - INFO - Loading data from `part_dimension` (SparkDataSet)...
2019-06-29 20:23:14,791 - kedro.io.data_catalog - INFO - Loading data from `supplier_dimension` (SparkDataSet)...
2019-06-29 20:23:14,965 - kedro.io.data_catalog - INFO - Loading data from `order_fact` (SparkDataSet)...


In [3]:
customer_dimension.registerTempTable("customer_dimension")
part_dimension.registerTempTable("part_dimension")
supplier_dimension.registerTempTable("supplier_dimension")
order_fact.registerTempTable("order_fact")

# Initiate Spark Session

In [4]:
spark = SparkSession \
    .builder \
    .getOrCreate()

# What are the top 5 nations in terms of revenue?

In [33]:
df=spark.sql("SELECT SD.Supplier_Country_Name\
                    ,SUM(OR.Order_Line_Revenue) AS Supplier_Country_Revenue\
                FROM order_fact OR \
          INNER JOIN supplier_dimension SD\
                  ON OR.Order_Supplier_Key=SD.Supplier_Key\
            GROUP BY SD.Supplier_Country_Name\
            ORDER BY Supplier_Country_Revenue DESC").limit(5)
df.show(5)

+---------------------+------------------------+
|Supplier_Country_Name|Supplier_Country_Revenue|
+---------------------+------------------------+
|        UNITED STATES|    1.7302219152000004E8|
|                CHINA|    1.5443322854000002E8|
|           MOZAMBIQUE|    1.4746607397999978E8|
|              VIETNAM|    1.2989473344999981E8|
|                EGYPT|    1.2832393222000003E8|
+---------------------+------------------------+



# From the top 5 nations, what is the most common shipping mode?

In [34]:
df.registerTempTable("Top_5_Nations")

In [35]:
df_ship=spark.sql("SELECT OR.Order_Ship_Mode\
                    ,COUNT(DISTINCT OR.Order_Customer_Key) AS Shipping_Count \
                FROM order_fact OR \
          INNER JOIN supplier_dimension SD\
                  ON OR.Order_Supplier_Key=SD.Supplier_Key\
               WHERE SD.Supplier_Country_Name IN (SELECT Supplier_Country_Name FROM Top_5_Nations) \
            GROUP BY OR.Order_Ship_Mode\
            ORDER BY Shipping_Count DESC").limit(1)
df_ship.show(1)

+---------------+--------------+
|Order_Ship_Mode|Shipping_Count|
+---------------+--------------+
|            FOB|           911|
+---------------+--------------+



# What are the top selling months?

In [36]:
df_sell=spark.sql("SELECT YEAR(Order_Date)*100 + MONTH(Order_Date) AS Year_Month\
                         ,SUM(Order_Line_Revenue) AS Monthly_Revenue\
                     FROM order_fact\
                 GROUP BY Year_Month\
                 ORDER BY Monthly_Revenue DESC")
df_sell.show(10)

+----------+--------------------+
|Year_Month|     Monthly_Revenue|
+----------+--------------------+
|    199310|       3.117616533E7|
|    199312|3.1122497909999996E7|
|    199201|3.0878726890000023E7|
|    199608|3.0497227109999996E7|
|    199512| 3.043916234000001E7|
|    199401| 3.032269908999999E7|
|    199309|3.0234220019999996E7|
|    199405|3.0015042540000007E7|
|    199409|2.9820136109999992E7|
|    199203|2.9758230360000014E7|
+----------+--------------------+
only showing top 10 rows



# Who are the top customer in terms of revenue and/or quantity?

In [39]:
spark.sql("SELECT CD.Customer_Name\
                 ,SUM(OR.Order_Line_Revenue) AS Customer_Revenue\
             FROM order_fact OR \
       INNER JOIN customer_dimension CD\
               ON OR.Order_Customer_Key=CD.Customer_Key\
         GROUP BY CD.Customer_Name\
         ORDER BY Customer_Revenue DESC").show(5)

+------------------+-----------------+
|     Customer_Name| Customer_Revenue|
+------------------+-----------------+
|Customer#000001489|        5457263.4|
|Customer#000000214|        4742494.2|
|Customer#000000073|       4714752.19|
|Customer#000001396|4678182.200000001|
|Customer#000001246|       4676314.75|
+------------------+-----------------+
only showing top 5 rows



In [40]:
spark.sql("SELECT CD.Customer_Name\
                 ,SUM(OR.Order_Line_Quantity) AS Customer_Quantity\
             FROM order_fact OR \
       INNER JOIN customer_dimension CD\
               ON OR.Order_Customer_Key=CD.Customer_Key\
         GROUP BY CD.Customer_Name\
         ORDER BY Customer_Quantity DESC").show(5)

+------------------+-----------------+
|     Customer_Name|Customer_Quantity|
+------------------+-----------------+
|Customer#000001489|             3868|
|Customer#000001396|             3408|
|Customer#000000073|             3384|
|Customer#000000214|             3369|
|Customer#000000898|             3309|
+------------------+-----------------+
only showing top 5 rows



# Compare the sales revenue of on current period against previous period

In [42]:
df_revn=spark.sql("SELECT YEAR(Order_Date)*100 + MONTH(Order_Date) AS Year_Month\
                         ,SUM(Order_Line_Revenue) AS Monthly_Revenue\
                     FROM order_fact\
                 GROUP BY Year_Month")
df_revn.registerTempTable("Revenue")

In [49]:
spark.sql("SELECT *\
                 ,LAG(Monthly_Revenue,12)OVER(ORDER BY Year_Month) AS Last_12_Monthly_Revenue\
                 ,Monthly_Revenue-LAG(Monthly_Revenue,12)OVER(ORDER BY Year_Month) AS Difference_Between_Previous_Year_Monthly_Revenue\
             FROM Revenue\
          ").show()

+----------+--------------------+-----------------------+------------------------------------------------+
|Year_Month|     Monthly_Revenue|Last_12_Monthly_Revenue|Difference_Between_Previous_Year_Monthly_Revenue|
+----------+--------------------+-----------------------+------------------------------------------------+
|    199201|3.0878726890000023E7|                   null|                                            null|
|    199202|2.5019705100000005E7|                   null|                                            null|
|    199203|2.9758230360000014E7|                   null|                                            null|
|    199204|       2.805256204E7|                   null|                                            null|
|    199205|        2.78214501E7|                   null|                                            null|
|    199206|2.5229275750000007E7|                   null|                                            null|
|    199207|2.6655226179999992E7|    